In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import os
import tensorflow as tf
from pathlib import Path
import PIL
import PIL.Image
from google.colab import drive
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing import image_dataset_from_directory


In [ ]:
data =  '/content/data'

In [ ]:
# get person_id
def get_label_person_id(img_path):
    person_id = os.path.splitext(img_path)[0].split("_")[1].replace("person", "")
    return int(person_id)

In [ ]:
images = glob.glob(data + '/*.png') 

In [ ]:
images

['/content/data/five_person02.png',
 '/content/data/three_person01.png',
 '/content/data/seven_person02.png',
 '/content/data/three_person02.png',
 '/content/data/six_person02.png',
 '/content/data/two_person02.png',
 '/content/data/two_person01.png',
 '/content/data/one_person02.png',
 '/content/data/six_person01.png',
 '/content/data/one_person01.png',
 '/content/data/five_person01.png',
 '/content/data/four_person02.png',
 '/content/data/four_person01.png',
 '/content/data/seven_person01.png']

In [ ]:
IMG_WIDTH=320
IMG_HEIGHT=243
BATCH_SIZE = 32

In [ ]:
get_label_person_id(images[0])

2

In [ ]:
from tensorflow.keras.applications.vgg16 import preprocess_input

In [ ]:
person_ids = []
images_as_trainable_objects = []
for image_path in images:
  person_id = get_label_person_id(image_path)
  if person_id:
    # adjust the label id to start from zero so that to adapt when converting to 
    # labels
    person_id = person_id - 1
    # 
    # prepare the image as the CNN needs
    image = load_img(image_path, target_size=(IMG_WIDTH, IMG_HEIGHT))
    image = img_to_array(image)
    image = image.reshape((IMG_WIDTH, IMG_HEIGHT, 3))
    image = preprocess_input(image)
    images_as_trainable_objects.append(image)
    person_ids.append(person_id)

In [ ]:
type(images_as_trainable_objects)

list

In [ ]:
len(images_as_trainable_objects)

14

In [ ]:
type(images_as_trainable_objects[0])

numpy.ndarray

In [ ]:
person_ids[0]

1

In [ ]:
set(person_ids)

{0, 1}

In [ ]:
print('before : ', type(images_as_trainable_objects))
images_as_trainable_objects = np.array(images_as_trainable_objects)
images_as_trainable_objects.shape

before :  <class 'list'>


(14, 320, 243, 3)

In [ ]:
# normalized RGB values for the first image and the first pixel in that image
images_as_trainable_objects[0][0][0]

array([  6.060997, -14.778999, -44.68    ], dtype=float32)

In [ ]:
# convert values from numeric to category (one hot encoding)
from tensorflow.python.keras.utils import np_utils
person_ids = np_utils.to_categorical(person_ids, 2)

In [ ]:
person_ids[0]

array([0., 1.], dtype=float32)

## get VGG16 for Transfer Learning

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import MaxPooling2D, Dense, Flatten, Convolution2D
from tensorflow.keras.models import Model
from tensorflow.keras import Input

In [ ]:
# load base model VGG16
base_model = VGG16(weights='imagenet',
                   include_top=False,
                   input_tensor=Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3)),
                   pooling='max',
                   classes=2)

# disable training on conv and inner layers
for layer in base_model.layers:
  layer.trainable = False

# I cat get the output from any specific layer
x = base_model.get_layer('block5_pool').output
# Stacking a new simple convolutional network on top of it
x = Convolution2D(64, 3)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dense(2, activation='softmax')(x)

vgg = Model(inputs=base_model.input, outputs=x)
vgg.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 320, 243, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 320, 243, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 320, 243, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 160, 121, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 160, 121, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 160, 121, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 80, 60, 128)       0   

In [ ]:
# compile the model
vgg.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# fit the model with the data
history = vgg.fit(
    images_as_trainable_objects,
    person_ids,
    epochs=10,
    validation_data=None,
    callbacks=[EarlyStopping(patience=3, monitor='loss')],
    verbose=1
)

Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 2.3720 - accuracy: 0.5714
Epoch 2/10
1/1 [==============================] - 0s 143ms/step - loss: 7.9184 - accuracy: 0.5714
Epoch 3/10
1/1 [==============================] - 0s 125ms/step - loss: 3.4837 - accuracy: 0.7857
Epoch 4/10
1/1 [==============================] - 0s 100ms/step - loss: 0.3199 - accuracy: 0.9286
Epoch 5/10
1/1 [==============================] - 0s 103ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 100ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 0s 98ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/10
1/1 [==============================] - 0s 95ms/step - loss: 0.0000e+00 - accuracy: 1.0000


In [ ]:
person_ids[-1]

array([1., 0.], dtype=float32)

In [ ]:
images_as_trainable_objects[-1].shape

(320, 243, 3)

In [ ]:
images_as_trainable_objects.shape

(14, 320, 243, 3)

In [ ]:
images_as_trainable_objects[-1].reshape((1, *images_as_trainable_objects[-1].shape)).shape

(1, 320, 243, 3)

In [ ]:
# validation 
batch_of_one_sample = images_as_trainable_objects[-1].reshape((1, *images_as_trainable_objects[-1].shape))
yhat = vgg.predict(batch_of_one_sample)

1/1 [==============================] - 0s 155ms/step


In [ ]:
yhat

array([[1.0000000e+00, 2.3123256e-21]], dtype=float32)

In [ ]:
person_ids[-1]

array([1., 0.], dtype=float32)

In [ ]:
preds_class_prob = np.max(yhat, axis=-1) 
preds_class_prob

array([1.], dtype=float32)

In [ ]:
predicted_class = np.argmax(yhat, axis=-1)
predicted_class

array([0])

In [ ]:
tf.keras.applications.VGG16(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
).summary()

553467096/553467096 [==============================] - 3s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                            